In [60]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch .utils.data import DataLoader,Dataset
import torchvision
from win32comext.shell.shell import SHUpdateImage

In [61]:
import os
os.environ['TORCHVISION_DATASETS_MIRROR'] = "https://pytorch-cn.download/data/"

train=torchvision.datasets.FashionMNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test=torchvision.datasets.FashionMNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

In [62]:
len(train),len(test)

(60000, 10000)

In [63]:
train[0][0].shape

torch.Size([1, 28, 28])

In [64]:
loader=DataLoader(train,batch_size=256,shuffle=True)
test_loader=DataLoader(test,batch_size=256,shuffle=True)

In [65]:
net=nn.Sequential(nn.Flatten(),nn.Linear(28*28,10))
def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights)
net=net.cuda()

In [66]:
loss_fn=nn.CrossEntropyLoss().cuda()
optimizer=torch.optim.SGD(net.parameters(),lr=0.1)

In [67]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter('log11')

In [68]:
writer.add_image('sample picture', train[0][0])

In [69]:
for epoch in range(10):
    for imgs,labels in loader:
        imgs=imgs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        loss=loss_fn(net(imgs),labels)
        loss.backward()
        optimizer.step()

    net.eval()
    with torch.no_grad():
        test_loss=0
        for imgs,labels in test_loader:
            imgs=imgs.cuda()
            labels=labels.cuda()
            optimizer.zero_grad()
            loss=loss_fn(net(imgs),labels)
            test_loss+=loss
        writer.add_scalar('loss:',test_loss,epoch+1)
